# Hello BattMo  

---
* Time: run all first time: ~ 3.5 minutes
* Time: run second time: ~ 20 seconds
* Time: run and read step by step: ~ 30 minutes

---

Welcome to this hands-on tutorial where we’ll explore the basics of BattMo.jl — a powerful Julia package for simulating lithium-ion battery cells using physics-based models like the Doyle-Fuller-Newman (DFN) model. 

By the end of this tutorial, you’ll:

- Understand some basic features of BattMo.jl
- Run your first battery simulation
- Explore and visualize the output
- Learn how to tweak key parameters for custom behavior




Make sure you have selected your own environment at the right top of the notebook. If you have created it, but it doesn't show up as an option, try restarting VSCode and see if it appears. Let's import BattMo and some other packages into the kernel.

In [33]:
using BattMo, GLMakie, Jutul
GLMakie.activate!(inline=true)


SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up


MethodError: MethodError: no method matching pipe_writer(::VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)})
The applicable method may be too new: running in world age 38639, while current world is 38699.

Closest candidates are:
  pipe_writer(::VSCodeServer.IJuliaCore.IJuliaStdio) (method too new to be called from this world context.)
   @ VSCodeServer c:\Users\lorenah\.vscode\extensions\julialang.language-julia-1.149.2\scripts\packages\IJuliaCore\src\stdio.jl:16
  pipe_writer(!Matched::Pipe)
   @ Base stream.jl:781
  pipe_writer(!Matched::Base.ProcessChain)
   @ Base process.jl:37
  ...


## Part 1 - Run a simple simulation

Let's run a simple simulation first so we get an idea of the steps that it involves and then dive into each necessary step with more detail. The following script runs a simple constant current discharge simulation using default parameter sets and a built in plotting functionality. This first run can take some time as Julia compiles all functions and structs that it encounters. This compilation makes a second simulation very fast. Run the cell a second time to see the different in solving time.

In [2]:
# step 1: Load parameters
cell_parameters = load_cell_parameters(; from_default_set= "Chen2020")
cycling_protocol = load_cycling_protocol(; from_default_set= "CCDischarge")
simulation_settings = load_simulation_settings(; from_default_set = "P2D")
# simulation_settings[""]

# step 2: Instantiate a model setup
model_setup = LithiumIonBattery()

# step 3: Instantiate a Simulation object
sim = Simulation(model_setup, cell_parameters, cycling_protocol; simulation_settings);

# step 4: Solve the simulation
output = solve(sim);

# step 5: Simple plotting
plot_dashboard(output; plot_type="contour")

 while trying to print a failed Task notice; giving up


UndefVarError: UndefVarError: `load_cell_parameters` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Let's now dive deeper into each step!

#### Step 1 - Load parameters and settings

BattMo.jl structures its simulation inputs into two primary categories: Parameters and Settings. This distinction helps users differentiate between the physical characteristics of the battery system and the numerical configurations of the simulation.

**Parameters** represent the controllable variables in real-world experiments. They are further divided into:

- **Cell Parameters**: define the intrinsic properties of the battery cell, such as geometry and material characteristics.
- **Cycling Protocol Parameters**: specify how the cell is operated during a simulation.

**Settings** are used to configure numerical assumptions for solving equations and finding numerical solutions. They are further divided into:

- **Model Settings**: define numerical assumptions related to the battery model, such as diffusion methods or simplifications used in the simulation.
- **Simulation Settings**: define numerical assumptions specific to the simulation process, including time-stepping schemes and discretization precision:

**BattMo** stores cell parameters, cycling protocols and settings in a user-friendly JSON format to facilitate reuse. We can load the parameters directly from the default sets which can be very convenient for a quick parameter loading to test a simulation setup. In order to quickly see which default sets BattMo provides we can use the following function to print information about the sets available.

In [3]:
print_default_input_sets_info()

UndefVarError: UndefVarError: `print_default_input_sets_info` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

For our example, we load the cell parameter set from a NMC811 vs Graphite-SiOx cell whose parameters were determined in the [Chen 2020 paper](https://doi.org/10.1149/1945-7111/ab9050). We also load an example cycling protocol for a simple Constant Current Discharge. We can load them directly from the build in sets.

In [4]:
cell_parameters = load_cell_parameters(; from_default_set = "Chen2020");
cycling_protocol = load_cycling_protocol(; from_default_set = "CCDischarge");

UndefVarError: UndefVarError: `load_cell_parameters` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

This a quick way of testing a setup, but for the purpose of this workshop we would like to be able to see what a parameter set contains. Therefore, we'll retrieve the default parameter sets that BattMo provides and store them locally in a folder. We can do this by running the following script.

In [5]:
path = "."
folder_name = "default_sets"
generate_default_parameter_files(path, folder_name; force = true)

UndefVarError: UndefVarError: `generate_default_parameter_files` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

As we stored the default sets in our own folder, we can alter the default files if we want to and load the parameters from our dedicated folder.

In [6]:
cell_parameters = load_cell_parameters(; from_file_path = "default_sets/cell_parameters/Chen2020.json");
cycling_protocol = load_cycling_protocol(; from_file_path = "default_sets/cycling_protocols/CCDischarge.json");

UndefVarError: UndefVarError: `load_cell_parameters` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

#### Step 2 - Select a model setup

Next, we select the default Lithium-Ion Battery setup. A model can be thought as a mathematical implementation of the electrochemical and transport phenomena occuring in a real battery cell. The implementation consist of a system of partial differential equations and their corresponding parameters, constants and boundary conditions. The default Lithium-Ion Battery setup selected below contains the model settings to simulate a basic P2D model, where neither current collectors nor SEI growth are considered.

In [7]:
model_setup = LithiumIonBattery();

UndefVarError: UndefVarError: `LithiumIonBattery` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

The LithiumIonBattery constructor validates the model settings on the back ground. If the model setup is valid we can continue and create a Simulation object by passing the model setup, cell parameters and a cycling protocol.

#### Step 3 - Initiate simulation object

 A Simulation can be thought as a procedure to predict how the cell responds to the cycling protocol by solving the equations in the model using the cell parameters passed. We first prepare the simulation.

In [8]:
sim = Simulation(model_setup, cell_parameters, cycling_protocol);

UndefVarError: UndefVarError: `Simulation` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

The simulation object is only instantiated when the model setup is valid. We can see that the Simulation object also validates the parameters and settings on the back ground. Each parameter set is validated on whether they are sensible and complete. 

#### Step 4 - Solve simulation

When the Simulation object is valid we can solve the simulation by passing the object to the solve function. As Julia is a compiled language, the first time that we run a simulation it will take some time to compile the functions and structs that it encounters. This makes running a second simulation very fast. See the difference by running the script for a second time.

In [9]:
output = solve(sim);

UndefVarError: UndefVarError: `solve` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

#### Step 5 - Simple plotting

We can use some built in functions for quick plotting. The dashboard gives you a quick overview of some important ouput variables. You can choose to have interactive line plots where you can change the time step using a slider or contour plots that show the position and time in one plot.

In [10]:
plot_dashboard(output; plot_type="contour")

UndefVarError: UndefVarError: `plot_dashboard` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

As the line plot is an interactive plot and notebooks are not able to render interactive plots, we'll show it in a separate window. 

In [11]:
GLMakie.activate!(inline=false)

plot_dashboard(output; plot_type="line")

UndefVarError: UndefVarError: `GLMakie` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Part 2 - Handle Cell Parameters

To change cell parameters, we can modify the JSON files directly, or we can read them into objects in the script and modify them as Dictionaries. A loaded cell parameter set is a Dictionary-like object which come with additional handy functions. First, lets list the outermost keys of the cell parameters object.

In [12]:
cell_parameters = load_cell_parameters(; from_file_path = "default_sets/cell_parameters/Chen2020.json")
keys(cell_parameters)

UndefVarError: UndefVarError: `load_cell_parameters` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Now we access the Separator key.

In [13]:
cell_parameters["Separator"]

UndefVarError: UndefVarError: `cell_parameters` not defined in `Main`
Suggestion: add an appropriate import or assignment. This global was declared but not assigned.

We have a flat list of parameters and values for the separator. In other cases, a key might nest other dictionaries, which can be accessed using the normal dictionary notation. Lets for instance see the active material parameters of the negative electrode.

In [14]:
cell_parameters["NegativeElectrode"]["ActiveMaterial"]

UndefVarError: UndefVarError: `cell_parameters` not defined in `Main`
Suggestion: add an appropriate import or assignment. This global was declared but not assigned.

In addition to manipulating parameters as dictionaries, we provide additional handy attributes and functions. For instance, we can display all cell parameters:

In [15]:
cell_parameters

UndefVarError: UndefVarError: `cell_parameters` not defined in `Main`
Suggestion: add an appropriate import or assignment. This global was declared but not assigned.

However, there are many parameters, nested into dictionaries. Often, we are more interested in a specific subset of parameters. We can find a parameter with the search_parameter function. For example, we'd like to now how electrode related objects and parameters are named:

In [16]:
search_parameter(cell_parameters, "Electrode")

UndefVarError: UndefVarError: `search_parameter` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

The search function also accepts partial matches and it is case-insentive.

In [17]:
search_parameter(cell_parameters, "char")

UndefVarError: UndefVarError: `search_parameter` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Parameter that take single numerical values (e.g. real, integers, booleans) can be directly modified. Examples:

In [18]:
cell_parameters["PositiveElectrode"]["ElectrodeCoating"]["Thickness"] = 8.2e-5

UndefVarError: UndefVarError: `cell_parameters` not defined in `Main`
Suggestion: add an appropriate import or assignment. This global was declared but not assigned.

Some parameters are described as functions or arrays, since the parameter value depends on other variables. For instance the Open Circuit Potentials of the Active Materials depend on the lithium stoichiometry and temperature. When we're unsure about the type or meaning of a parameter, we can print information on invidual parameters as well. For some parameters, that require more explanation, a link to the documentation is provided. Visit the documentation of the OpenCircuitPotential parameter to find more information on how to implement you own user defined functional parameters.

In [19]:
parameter_name = "OpenCircuitPotential"

print_parameter_info(parameter_name)

UndefVarError: UndefVarError: `print_parameter_info` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Part 3 - Handle Cycling protocols

The cycling protocol parameters can be altered in the same way as the cell parameters. Let's load a default CCCV cycling protocol.

In [20]:
cycling_protocol = load_cycling_protocol(; from_file_path= "default_sets/cycling_protocols/CCCV.json")
cycling_protocol["UpperVoltageLimit"] = 4.1
cycling_protocol["LowerVoltageLimit"] = 2.6

UndefVarError: UndefVarError: `load_cycling_protocol` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

We can for example change the total number of cycles to 10 and change the CRate to 2.0.

In [21]:
cycling_protocol["TotalNumberOfCycles"] = 5
cycling_protocol["CRate"] = 0.5
cycling_protocol["DRate"] = 0.5

UndefVarError: UndefVarError: `cycling_protocol` not defined in `Main`
Suggestion: add an appropriate import or assignment. This global was declared but not assigned.

Let's run the CCCV protocol with the altered positive electrode coating thickness and plot some results

In [22]:
model_setup = LithiumIonBattery()

sim = Simulation(model_setup, cell_parameters, cycling_protocol)

output = solve(sim);

UndefVarError: UndefVarError: `LithiumIonBattery` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Let's use the simple dashboard plotting function to plot the voltage and current over time.

In [23]:
GLMakie.activate!(inline=true)
plot_dashboard(output; plot_type = "simple")

UndefVarError: UndefVarError: `GLMakie` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Part 4 - Retrieve output quantities and cell metrics

Let's use the output from the previous CCCV simulation. The output is a NamedTuple that contains a lot of different data and can sometimes be a bit difficult to navigate. Therefore BattMo provides some handy functions to retrieve the data that you want. First print an overview of the output to see which variables are available in the output.

In [24]:
print_output_overview(output)

UndefVarError: UndefVarError: `print_output_overview` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In the overview we can quickly see the aivalable variables and their units. We can also see that the variables have been devided into three categories: time series, states, and metrics. This has been done to provide some structure to the variables that is intuitive and cleans up the data. We also use these three categories when retrieving data. Let's first retrieve for example time series data like voltage, current and time.

In [25]:
time_series = get_output_time_series(output)

t = time_series[:Time]
E = time_series[:Voltage]
I = time_series[:Current];

UndefVarError: UndefVarError: `get_output_time_series` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Let's now retrieve some state variables

In [26]:
states = get_output_states(output)

negative_electrode_concentration = states[:NeAmConcentration]
negative_electrode_surface_concentration = states[:NeAmSurfaceConcentration];

UndefVarError: UndefVarError: `get_output_states` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

We can also print more information on each individual variable. For example, it might be a bit unclear what the difference is between the NeAmConcenctration and NeAmSurfaceConcentration. For this purpose we can use the `print_output_variable_info()` function.

In [27]:
print_output_variable_info("NeAmConcentration")

UndefVarError: UndefVarError: `print_output_variable_info` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [28]:
print_output_variable_info("NeAmSurfaceConcentration")

UndefVarError: UndefVarError: `print_output_variable_info` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

We can have a better idea of what the variable represents by reading the description and checking the shape of the variable. We can also retrieve some KPIs from the output.

In [29]:
metrics = get_output_metrics(output)

discharge_capacity = metrics[:DischargeCapacity]
round_trip_efficiency = metrics[:RoundTripEfficiency]
cycle_numbers = metrics[:CycleNumber]

UndefVarError: UndefVarError: `get_output_metrics` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

We can see that the last time step step contains cycle number 10 which represents the start of the 11th cycle, but as the simulation was ordered to complete 10 cycles, it terminates right after entering the 11th cycle. Therefore, we'll remove the data for cycle 10. Let's retrieve the unique cycle numbers in an array and plot the discharge capacity against it.

In [30]:
unique_cycle_array = unique(cycle_numbers) .+ 1  # Remove last cycle number, and shift so that the cycle numbers start from 1.

f = Figure(size = (1000, 400))

ax = Axis(f[1, 1], title = "Round trip efficiency", xlabel = "Cycle number / -", ylabel = "Efficiency / %",
)
scatterlines!(ax, unique_cycle_array, round_trip_efficiency; linewidth = 4)

ax = Axis(f[2, 1], title = "Discharge capacity", xlabel = "Cycle number / -", ylabel = "Capacity / Ah",
)
scatterlines!(ax, unique_cycle_array, discharge_capacity; linewidth = 4)

f

UndefVarError: UndefVarError: `cycle_numbers` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [31]:
#length(unique_cycle_array)
model_setup

UndefVarError: UndefVarError: `model_setup` not defined in `Main`
Suggestion: add an appropriate import or assignment. This global was declared but not assigned.

## Summary and overview

We learned how to run a simple simulation using default parameters and settings.

In [32]:
using BattMo, GLMakie

cell_parameters = load_cell_parameters(; from_file_path = "default_sets/cell_parameters/Chen2020.json")
cycling_protocol = load_cycling_protocol(; from_file_path = "default_sets/cycling_protocols/CCDischarge.json")

model_setup = LithiumIonBattery()

sim = Simulation(model_setup, cell_parameters, cycling_protocol);

output = solve(sim);

plot_dashboard(output)


SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up


MethodError: MethodError: no method matching pipe_writer(::VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)})
The applicable method may be too new: running in world age 38639, while current world is 38699.

Closest candidates are:
  pipe_writer(::VSCodeServer.IJuliaCore.IJuliaStdio) (method too new to be called from this world context.)
   @ VSCodeServer c:\Users\lorenah\.vscode\extensions\julialang.language-julia-1.149.2\scripts\packages\IJuliaCore\src\stdio.jl:16
  pipe_writer(!Matched::Pipe)
   @ Base stream.jl:781
  pipe_writer(!Matched::Base.ProcessChain)
   @ Base process.jl:37
  ...
